# Comprendre le Mécanisme d'Attention

**Note de synthèse**

---

## L'idée fondamentale

L'attention répond à une question simple :

> **"Pour comprendre ce mot, quels autres mots dois-je regarder ?"**

Dans la phrase *"Le chat qui dormait sur le canapé a sauté"*, pour comprendre "a sauté", il faut regarder "chat" (le sujet), pas "canapé".

L'attention permet à chaque mot de **récupérer de l'information** des autres mots de manière **pondérée**.

---

## La formule de l'attention

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

### D'où vient cette formule ?

Elle est construite étape par étape pour répondre au besoin :

| Étape | Opération | Pourquoi |
|-------|-----------|----------|
| 1 | $Q \cdot K^T$ | Mesurer la **compatibilité** entre mots (produit scalaire) |
| 2 | $\div \sqrt{d_k}$ | **Stabiliser** les valeurs (éviter que softmax sature) |
| 3 | softmax | Transformer en **poids** positifs qui somment à 1 |
| 4 | $\times V$ | **Récupérer** l'information pondérée |

C'est une façon **différentiable** de dire : "regarde les autres mots et récupère l'info pertinente".

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import math

def scaled_dot_product_attention(Q, K, V):
    """
    Implémentation de l'attention.
    
    Args:
        Q: Queries (seq_len, d_k)
        K: Keys (seq_len, d_k)
        V: Values (seq_len, d_v)
    
    Returns:
        output: Résultat de l'attention
        weights: Poids d'attention
    """
    d_k = K.shape[-1]
    
    # Étape 1 : Compatibilité entre chaque paire de mots
    scores = Q @ K.transpose(-2, -1)
    
    # Étape 2 : Stabilisation
    scores = scores / math.sqrt(d_k)
    
    # Étape 3 : Poids normalisés
    weights = F.softmax(scores, dim=-1)
    
    # Étape 4 : Récupération pondérée
    output = weights @ V
    
    return output, weights

---

## Q, K, V : Qu'est-ce que c'est ?

### Les noms

| Notation | Nom complet | Intuition |
|----------|-------------|----------|
| **Q** | Query | Ce que le mot **cherche** |
| **K** | Key | Comment le mot **se présente** aux autres |
| **V** | Value | L'**information** que le mot transmet |

### Attention : c'est une métaphore !

Les matrices W_q, W_k, W_v n'ont pas "conscience" d'être des queries, keys ou values. Ces noms sont une **analogie** pour comprendre l'architecture, pas une description de ce que les matrices apprennent.

### D'où viennent Q, K, V ?

Ils sont calculés à partir des **embeddings** via des matrices apprenables :

```
x (embedding du mot)
    │
    ├──► W_q @ x ──► Q
    ├──► W_k @ x ──► K  
    └──► W_v @ x ──► V
```

W_q, W_k, W_v sont des matrices de **projection** initialisées aléatoirement, puis ajustées pendant l'entraînement.

---

## Les dimensions : clarification

### Trois concepts différents

| Concept | Notation | Exemple | C'est quoi |
|---------|----------|---------|------------|
| **Vocabulaire** | vocab_size | 50 000 | Nombre total de mots/tokens distincts |
| **Dimension d'embedding** | d_model | 512 | Taille du vecteur représentant UN mot |
| **Longueur de séquence** | seq_len | 10 | Nombre de mots dans la phrase |

### Pour UN mot

```
embedding de "chat" : [0.1, 0.3, ..., 0.2]   ← 512 nombres (d_model)
Q de "chat"         : [0.4, 0.1, ..., 0.8]   ← 512 nombres (d_model)
K de "chat"         : [0.2, 0.5, ..., 0.3]   ← 512 nombres (d_model)
V de "chat"         : [0.7, 0.2, ..., 0.1]   ← 512 nombres (d_model)
```

Chaque mot a UN vecteur embedding, UNE query, UNE key, UNE value. Tous de même dimension (d_model).

### Pour une phrase de 3 mots

On empile les vecteurs :

```
X (embeddings) : shape (3, 512)
Q (queries)    : shape (3, 512)   ← 3 queries de dim 512
K (keys)       : shape (3, 512)   ← 3 keys de dim 512
V (values)     : shape (3, 512)   ← 3 values de dim 512
```

### Où intervient le vocabulaire ?

Uniquement au **début** (embedding lookup) et à la **fin** (projection vers le vocabulaire) du modèle. L'attention ne voit **jamais** le vocabulaire directement.

In [ ]:
# Démonstration des dimensions
vocab_size = 50000  # Nombre de mots distincts
d_model = 512       # Dimension d'un embedding
seq_len = 3         # Nombre de mots dans la phrase

# Simulation
X = torch.randn(seq_len, d_model)  # 3 embeddings de dim 512

W_q = nn.Linear(d_model, d_model, bias=False)
W_k = nn.Linear(d_model, d_model, bias=False)
W_v = nn.Linear(d_model, d_model, bias=False)

Q = W_q(X)  # (3, 512)
K = W_k(X)  # (3, 512)
V = W_v(X)  # (3, 512)

print(f"Vocabulaire     : {vocab_size} mots distincts")
print(f"Dimension (d)   : {d_model}")
print(f"Séquence        : {seq_len} mots")
print()
print(f"X shape : {X.shape}  ← {seq_len} embeddings de dim {d_model}")
print(f"Q shape : {Q.shape}  ← {seq_len} queries de dim {d_model}")
print(f"K shape : {K.shape}  ← {seq_len} keys de dim {d_model}")
print(f"V shape : {V.shape}  ← {seq_len} values de dim {d_model}")

---

## Q, K, V concernent quels mots ?

**Tous les mots à la fois.**

Pour la phrase "Le chat mange" :

In [ ]:
# Exemple concret
mots = ["Le", "chat", "mange"]
seq_len = len(mots)
d_model = 8  # Petite dimension pour l'exemple

torch.manual_seed(42)
X = torch.randn(seq_len, d_model)
W_q = nn.Linear(d_model, d_model, bias=False)
W_k = nn.Linear(d_model, d_model, bias=False)
W_v = nn.Linear(d_model, d_model, bias=False)

Q = W_q(X)
K = W_k(X)
V = W_v(X)

print("Chaque mot a sa propre Query, Key, Value :")
print("=" * 50)
for i, mot in enumerate(mots):
    print(f"\n{mot}:")
    print(f"  Q[{i}] = {Q[i][:4].tolist()}...  (ce que '{mot}' cherche)")
    print(f"  K[{i}] = {K[i][:4].tolist()}...  (comment '{mot}' se présente)")
    print(f"  V[{i}] = {V[i][:4].tolist()}...  (info que '{mot}' transmet)")

---

## Le produit Q @ K.T : que calcule-t-on ?

Le produit `Q @ K.T` donne une matrice de **scores de compatibilité** entre chaque paire de mots.

```
scores[i, j] = Q[i] · K[j] = "À quel point le mot i est attentif au mot j"
```

In [ ]:
# Calcul des scores
scores = Q @ K.T

print("Matrice des scores (Q @ K.T) :")
print("=" * 50)
print()
print(f"{'':>10}", end="")
for mot in mots:
    print(f"{mot:>10}", end="")
print("   ← Keys (comment ils se présentent)")
print()

for i, mot in enumerate(mots):
    print(f"{mot:>10}", end="")
    for j in range(len(mots)):
        print(f"{scores[i,j].item():>10.2f}", end="")
    if i == 0:
        print("   ← Queries")
    elif i == 1:
        print("      (ce qu'ils cherchent)")
    else:
        print()

print()
print("Lecture : scores[1,2] = compatibilité entre Q['chat'] et K['mange']")

In [ ]:
# Application de l'attention complète
output, weights = scaled_dot_product_attention(Q, K, V)

print("Poids d'attention (après softmax) :")
print("=" * 50)
print()

for i, mot in enumerate(mots):
    print(f"{mot} regarde : ", end="")
    for j, mot_j in enumerate(mots):
        poids = weights[i, j].item()
        bar = "█" * int(poids * 20)
        print(f"{mot_j}({poids:.2f}){bar}  ", end="")
    print()

In [ ]:
# Visualisation
plt.figure(figsize=(6, 5))
plt.imshow(weights.detach().numpy(), cmap='Blues')
plt.xticks(range(len(mots)), mots)
plt.yticks(range(len(mots)), mots)
plt.xlabel("Mots regardés (Keys)")
plt.ylabel("Mots qui regardent (Queries)")
plt.title("Poids d'attention")
plt.colorbar(label="Poids")

for i in range(len(mots)):
    for j in range(len(mots)):
        val = weights[i, j].item()
        color = 'white' if val > 0.5 else 'black'
        plt.text(j, i, f'{val:.2f}', ha='center', va='center', color=color)

plt.tight_layout()
plt.show()

---

## Pourquoi 3 matrices différentes (W_q, W_k, W_v) ?

### Pourquoi ne pas faire Q = K = V = x ?

Si on utilisait directement les embeddings sans projection :

```
scores = x @ x.T
```

On calculerait les **similarités sémantiques brutes** entre mots. "Chat" serait attentif à "chien", "félin", etc.

Mais ce qu'on veut souvent, c'est capturer des relations **syntaxiques** (sujet-verbe) ou **contextuelles**, pas seulement sémantiques.

### Avec 3 matrices différentes

Les projections permettent de transformer les embeddings dans des espaces où d'**autres types de relations** deviennent visibles.

Le modèle peut apprendre :
- W_q : projeter pour "chercher" certains patterns
- W_k : projeter pour "être trouvé" par certains patterns
- W_v : projeter pour "transmettre" certaines informations

Ces rôles ne sont pas programmés, ils **émergent** de l'entraînement.

---

## Comment les matrices apprennent-elles ?

### Ce n'est pas magique !

Les matrices W_q, W_k, W_v sont ajustées par **backpropagation**, comme tous les autres poids du réseau.

### L'architecture comme "moule"

L'idée clé :

1. **On définit une architecture** avec une contrainte structurelle :
   ```
   output = softmax(Q @ K.T / √d) @ V
   ```

2. **Les matrices sont dans un "moule"** : Q est toujours à gauche, K toujours transposé à droite, V toujours à la fin.

3. **L'entraînement ajuste les matrices** pour que cette formule produise des résultats utiles pour la tâche.

4. **Les propriétés émergent** : les matrices acquièrent des propriétés qui "fonctionnent" dans leur position.

### Analogie

C'est comme une **clé et une serrure** :
- La forme de la clé (W_q) et la forme de la serrure (W_k) s'ajustent **ensemble**
- On ne dit pas que la clé "apprend à être une clé"
- Mais sa forme s'affine pour fonctionner avec la serrure

Le "rôle" de Q, K, V est imposé par leur **position dans la formule**, pas par ce qu'ils "savent".

---

## L'attention dans le modèle complet

### L'attention n'a pas de loss propre !

L'attention est une **brique** dans un pipeline plus grand. La loss est calculée à la **fin** du modèle.

### Flux complet (exemple : prédire le mot suivant)

```
"Le chat mange" (input)
       ↓
   Token IDs : [42, 1337, 856]
       ↓
   Embedding lookup (matrice vocab_size × d_model)
       ↓
   Embeddings : (3, 512)
       ↓
   + Positional Encoding
       ↓
┌─────────────────────────────────┐
│  Bloc Transformer (× N fois)   │
│  ┌───────────────────────────┐ │
│  │ Self-Attention            │ │  ← Q, K, V sont ici
│  └───────────────────────────┘ │
│  ┌───────────────────────────┐ │
│  │ Feed-Forward              │ │
│  └───────────────────────────┘ │
└─────────────────────────────────┘
       ↓
   Projection (d_model → vocab_size)
       ↓
   Softmax → Probabilités sur le vocabulaire
       ↓
   Prédiction : "la" (mot le plus probable)
```

### Comment ça s'entraîne

```
1. Forward : "Le chat mange" → modèle → P("la") = 0.1

2. Loss : -log(P("la")) = -log(0.1) = 2.3  (erreur élevée)

3. Backpropagation :
   Loss → ∂/∂(projection) → ∂/∂(FFN) → ∂/∂(W_v) → ∂/∂(W_k) → ∂/∂(W_q) → ...

4. Mise à jour :
   W_q = W_q - lr × ∂Loss/∂W_q
   W_k = W_k - lr × ∂Loss/∂W_k
   W_v = W_v - lr × ∂Loss/∂W_v
```

Les gradients de la loss **remontent** jusqu'aux matrices d'attention.

---

## Self-Attention vs Cross-Attention

### Self-Attention (ce qu'on a vu)

Q, K, V viennent de la **même** séquence :

```python
Q = W_q @ X  # X = embeddings de "Le chat mange"
K = W_k @ X  # même X
V = W_v @ X  # même X
```

Chaque mot regarde les autres mots **de la même phrase**.

### Cross-Attention (traduction, etc.)

Q vient d'une séquence, K et V d'une **autre** :

```python
# Traduction anglais → français
Q = W_q @ X_français  # "Le chat mange"
K = W_k @ X_anglais   # "The cat eats"
V = W_v @ X_anglais   # "The cat eats"
```

Le décodeur français "interroge" l'encodeur anglais pour savoir quels mots anglais regarder.

### Cas d'usage

| Type | Usage |
|------|-------|
| **Self-Attention** | GPT, BERT, comprendre une phrase |
| **Cross-Attention** | Traduction, question-réponse, image captioning |

---

## Pourquoi diviser par √d_k ?

### Le problème

Quand d_k est grand, les produits scalaires Q·K deviennent grands (en valeur absolue).

Softmax sur des grandes valeurs → distribution très "piquée" (une valeur proche de 1, les autres proches de 0).

Conséquence : gradients très petits → apprentissage bloqué.

In [ ]:
# Démonstration : effet du scaling
d_k = 512  # Grande dimension

torch.manual_seed(42)
Q = torch.randn(5, d_k)
K = torch.randn(5, d_k)

# Sans scaling
scores_sans = Q @ K.T
weights_sans = F.softmax(scores_sans, dim=-1)

# Avec scaling
scores_avec = (Q @ K.T) / math.sqrt(d_k)
weights_avec = F.softmax(scores_avec, dim=-1)

print("SANS SCALING :")
print(f"  Scores : min={scores_sans.min():.1f}, max={scores_sans.max():.1f}")
print(f"  Poids max par ligne : {weights_sans.max(dim=-1).values.tolist()}")
print(f"  → Distribution très piquée (proches de 1)")

print("\nAVEC SCALING (÷ √512 ≈ ÷ 22.6) :")
print(f"  Scores : min={scores_avec.min():.1f}, max={scores_avec.max():.1f}")
print(f"  Poids max par ligne : {weights_avec.max(dim=-1).values.tolist()}")
print(f"  → Distribution plus douce, meilleur apprentissage")

---

## Récapitulatif

| Question | Réponse |
|----------|--------|
| **Que fait l'attention ?** | Permet à chaque mot de récupérer de l'info des autres mots |
| **D'où vient la formule ?** | Construction pour mesurer compatibilité (Q·K), normaliser (softmax), récupérer (×V) |
| **Q, K, V c'est quoi ?** | Projections des embeddings via matrices apprenables |
| **Pourquoi 3 matrices ?** | Pour projeter dans des espaces différents (pas juste similarité sémantique) |
| **Comment ça apprend ?** | Backpropagation depuis la loss finale du modèle |
| **Dimensions ?** | Q, K, V : (seq_len, d_model). Le vocabulaire n'intervient pas. |
| **√d_k ?** | Stabilise les scores pour éviter que softmax sature |

### L'essentiel

L'attention est une **brique différentiable** qui permet aux mots de "communiquer". Les matrices W_q, W_k, W_v sont ajustées pendant l'entraînement pour que cette communication soit **utile** pour la tâche finale.

Le "rôle" de Q, K, V (questions, clés, valeurs) est une **métaphore** pour comprendre l'architecture. En réalité, c'est la **position dans la formule** qui détermine leur fonction, et l'entraînement qui affine leurs propriétés.